In [ ]:
#Data store on google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/DataScienceProject.zip" -d "/content/"

Archive:  /content/drive/MyDrive/DataScienceProject.zip
   creating: /content/DataScienceProject/benign/
  inflating: /content/DataScienceProject/benign/benign (179).png  
  inflating: /content/DataScienceProject/benign/benign (18).png  
  inflating: /content/DataScienceProject/benign/benign (180).png  
  inflating: /content/DataScienceProject/benign/benign (181).png  
  inflating: /content/DataScienceProject/benign/benign (182).png  
  inflating: /content/DataScienceProject/benign/benign (183).png  
  inflating: /content/DataScienceProject/benign/benign (184).png  
  inflating: /content/DataScienceProject/benign/benign (185).png  
  inflating: /content/DataScienceProject/benign/benign (186).png  
  inflating: /content/DataScienceProject/benign/benign (187).png  
  inflating: /content/DataScienceProject/benign/benign (188).png  
  inflating: /content/DataScienceProject/benign/benign (189).png  
  inflating: /content/DataScienceProject/benign/benign (19).png  
  inflating: /content/Data

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from pandas.core.common import flatten
import copy
import numpy as np
import random
import pandas as pd

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim # optimzer


import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

import glob
from tqdm import tqdm

In [ ]:
def ResizeImage(img):
    #img = cv2.imread('D:\CancerImages\DataScienceProject/train/benign (2).png', cv2.IMREAD_UNCHANGED)
   
    #print('Original Dimensions : ',img.shape)
 
    width = 256
    height = 256
    dim = (width, height)
  
# resize image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
 
    #print('Resized Dimensions : ',resized.shape)
 
    #plt.imshow(resized)
    return resized


In [ ]:
#######################################################
#               Define Transforms
#######################################################

#To define an augmentation pipeline, you need to create an instance of the Compose class.
#As an argument to the Compose class, you need to pass a list of augmentations you want to apply. 
#A call to Compose will return a transform function that will perform image augmentation.
#(https://albumentations.ai/docs/getting_started/image_augmentation/)

train_transforms = A.Compose(
    [
        A.SmallestMaxSize(max_size=350),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=360, p=0.5),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.MultiplicativeNoise(multiplier=[0.5,2], per_channel=True, p=0.2),
        A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
        ToTensorV2(),
    ]
)

test_transforms = A.Compose(
    [
        A.SmallestMaxSize(max_size=350),
        ToTensorV2(),
    ]
)

In [ ]:
train_data_path = '/content/DataScienceProject/train'


In [ ]:
train_image_paths = [] #to store image paths in list

#1.
# get all the paths from train_data_path and append image paths and class to to respective lists
# eg. train path-> 'images/train/26.Pont_du_Gard/4321ee6695c23c7b.jpg'
# eg. class -> 26.Pont_du_Gard
#for data_path in glob.glob(train_data_path + '/*.png'):
train_image_paths.append(glob.glob(train_data_path + '/*'))
    
train_image_paths = list(flatten(train_image_paths))
random.shuffle(train_image_paths)

print('train_image_path example: ', train_image_paths[0])
#print('class example: ', classes[0])

#2.
# split train valid from train paths (80,20)
#train_image_paths, valid_image_paths = train_image_paths[:int(0.8*len(train_image_paths))], train_image_paths[int(0.8*len(train_image_paths)):] 

#3.
# create the test_image_paths


print("Train size: {}\n".format(len(train_image_paths)))



train_image_path example:  /content/DataScienceProject/train/malignant (48).png
Train size: 278



In [ ]:
normal_image_path=[]
normal_test_path=[]
normal_image_path=(glob.glob("/content/DataScienceProject/normal/*.png"))
normal_test_path=(glob.glob("/content/DataScienceProject/test_normal/*.png"))
len(normal_image_path)

113

In [ ]:
test_image_path=[]
test_image_path=glob.glob("/content/DataScienceProject/test/*.png")
len(test_image_path)

27

In [ ]:
train=[]
test=[]
for path in train_image_paths:
    img=plt.imread(path)
    train.append((ResizeImage(img),1))
for path in normal_image_path:
    img=plt.imread(path)
    train.append((ResizeImage(img),0))

for path in test_image_path:
    img=plt.imread(path)
    test.append((ResizeImage(img),1))
for path in normal_test_path:
    img=plt.imread(path)
    test.append((ResizeImage(img),0))

    


In [ ]:
len(test)

47

In [ ]:
random.shuffle(test)

In [ ]:
class BreastCancer(Dataset):
    def __init__(self, image_paths, transform=False):
        self.image_paths = image_paths
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_fi = self.image_paths[idx][0]
        print(image_fi)
        image = cv2.cvtColor(image_fi, cv2.COLOR_BGR2RGB)
        
        label = self.image_paths[idx][1]
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        
        return image, label

In [ ]:
random.shuffle(train)

    

In [ ]:
train_dataset = BreastCancer(train,train_transforms)


In [ ]:
test_dataset = BreastCancer(test,test_transforms)

In [ ]:
train_loader = DataLoader(
    train_dataset, batch_size=20, shuffle=True
)
test_loader = DataLoader(
    test_dataset, batch_size=8, shuffle=True
)

In [ ]:
classes = ['No','Yes'] #to store class values


In [ ]:
class Net(nn.Module):
    #Models a simple Convolutional Neural Network
	
    def __init__(self):
	#initialize the network '''
         super(Net, self).__init__()
	# 3 input image channel, 6 output channels, 
	# 5x5 square convolution kernel
         self.conv1 = nn.Conv2d(in_channels=3,out_channels=24,kernel_size=3,padding=2)
         self.bn1=nn.BatchNorm2d(num_features=24)
         self.relu1=nn.ReLU()
         self.pool1=nn.MaxPool2d(kernel_size=2)

   #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
         self.conv2=nn.Conv2d(in_channels=24,out_channels=18,kernel_size=3)
         self.bn2=nn.BatchNorm2d(num_features=18)
         self.relu2=nn.ReLU()
         self.pool2=nn.MaxPool2d(kernel_size=2)

         
         self.conv3=nn.Conv2d(in_channels=18,out_channels=12,kernel_size=3)
         self.bn3=nn.BatchNorm2d(num_features=12)
         self.relu3=nn.ReLU()
         self.pool3=nn.MaxPool2d(kernel_size=2)


         self.conv4=nn.Conv2d(in_channels=12,out_channels=10,kernel_size=3)
         self.bn4=nn.BatchNorm2d(num_features=10)
         self.relu4=nn.ReLU()
         self.pool4=nn.MaxPool2d(kernel_size=2)



         self.conv5=nn.Conv2d(in_channels=10,out_channels=32,kernel_size=3)
         self.bn5=nn.BatchNorm2d(num_features=32)
         self.relu5=nn.LeakyReLU()
         self.pool5=nn.MaxPool2d(kernel_size=2)
       
       

         self.fc=nn.Linear(in_features=256 * 256 * 32,out_features=2)
         self.final = nn.Sigmoid()

    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        output=self.pool1(output)  #pooling with kernal size 2
            
        output=self.conv2(output)
        output=self.relu2(output)
        output=self.pool2(output)  

            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
        output=self.pool3(output)  


        output=self.conv4(output)
        output=self.bn4(output)
        output=self.relu4(output)
        output=self.pool4(output)  

        output=self.conv5(output)
        output=self.bn5(output)
        output=self.relu5(output)
        output=self.pool5(output)  

         


            
        output = F.interpolate(output, size=(256, 256))  

            #Above output will be in matrix form, with shape (256,32,75,75)
           # x = x.view(x.size(0), -1)

        output=output.view(output.size(0),-1)
            
            # output=output.view(output.size(0),,32*256*256)

        output=self.fc(output)
            
        return output

In [ ]:
model = Net()
model

Net(
  (conv1): Conv2d(3, 24, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
  (bn1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(24, 18, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(18, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(18, 12, kernel_size=(3, 3), stride=(1, 1))
  (bn3): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(12, 10, kernel_size=(3, 3), stride=(1, 1))
  (bn4): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu4): ReLU()
  (pool4): MaxPool2d(kernel_size=2, stride=2, padd

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0

# whatever you are timing goes here

# Waits for everything to finish running
#torch.cuda.synchronize()
print('Finished Training')


Streaming output truncated to the last 5000 lines.
  [0.09003641 0.09003641 0.09003641]]]
[[[0.5626552  0.5626552  0.5626552 ]
  [0.48633945 0.48633945 0.48633945]
  [0.706738   0.706738   0.706738  ]
  ...
  [0.6528837  0.6528837  0.6528837 ]
  [0.39241895 0.39241895 0.39241895]
  [0.00718926 0.00718926 0.00718926]]

 [[0.05380934 0.05380934 0.05380934]
  [0.12423447 0.12423447 0.12423447]
  [0.46911806 0.46911806 0.46911806]
  ...
  [0.47220635 0.47220635 0.47220635]
  [0.44750202 0.44750202 0.44750202]
  [0.0449962  0.0449962  0.0449962 ]]

 [[0.01803947 0.01803947 0.01803947]
  [0.04277805 0.04277805 0.04277805]
  [0.33876538 0.33876538 0.33876538]
  ...
  [0.50114894 0.50114894 0.50114894]
  [0.5102557  0.5102557  0.5102557 ]
  [0.2430213  0.2430213  0.2430213 ]]

 ...

 [[0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  ...
  [0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  [0.00392157 0.0039215

In [ ]:
PATH = "/content/entire_model2_60pc_5l_avgpooling_2epoch.pt"
# Save
torch.save(model, PATH)

In [ ]:
PATH='entire_model1.pt'
#torch.save(model,PATH)

In [ ]:
model = torch.load(PATH)
model.eval()

RuntimeError: ignored

In [ ]:
correct = 0
total = 0
i=0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        print(labels)
        # calculate outputs by running images through the network
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        print(predicted)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

[[[0.9051604  0.9051604  0.9051604 ]
  [0.8579265  0.8579265  0.8579265 ]
  [0.84708863 0.84708863 0.84708863]
  ...
  [0.7118906  0.7118906  0.7118906 ]
  [0.6584448  0.6584448  0.6584448 ]
  [0.64531606 0.64531606 0.64531606]]

 [[0.8759173  0.8759173  0.8759173 ]
  [0.8305478  0.8305478  0.8305478 ]
  [0.834386   0.834386   0.834386  ]
  ...
  [0.6552861  0.6552861  0.6552861 ]
  [0.6380251  0.6380251  0.6380251 ]
  [0.65028286 0.65028286 0.65028286]]

 [[0.6843187  0.6843187  0.6843187 ]
  [0.69969666 0.69969666 0.69969666]
  [0.7495439  0.7495439  0.7495439 ]
  ...
  [0.58999705 0.58999705 0.58999705]
  [0.57582396 0.57582396 0.57582396]
  [0.59602094 0.59602094 0.59602094]]

 ...

 [[0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  ...
  [0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]]

 [[0.00392157 0.00392157 0.00392157]
  [0.00392157 0.00392157 0.00392157]
